In [3]:
import itertools
import pickle 
import networkx as nx
from collections import deque, defaultdict, Counter
from tqdm import tqdm
import community as community_louvain
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import regex as re
import math
import nltk

In [5]:
with open("../base_graph.pkl", 'rb') as f:
    G_directed = pickle.load(f)
f.close()
G = G_directed.to_undirected()

In [ ]:
with open("partition.pkl", 'rb') as f:
    partition = pickle.load(f)
    
with open("mod.pkl", 'rb') as f:
    mod = pickle.load(f)  
    
number_of_communities = len(set(partition.values()))


In [ ]:
'''
community_dict[2] = [['27c5ea64-86cb-4e69-9d13-c8ba2654515d'],
 ['2ee9a087-6188-4ebd-95b9-6561cba0584c'],
 ['efe2dd1d-706c-4ab6-bd9b-90d35a81d04f']]
'''

community_dict = {new_list: [] for new_list in range(number_of_communities)}
for i, j in partition.items():  
    community_dict[j].append([i])
    
# Filter out communities with only one element
community_dict_bigger_than_one = {k: v for k, v in community_dict.items() if len(v) > 1}

In [ ]:
community_size = np.zeros(number_of_communities)

for i,j in enumerate(community_dict):
    community_size[i] = (len(community_dict[j]))
    
hist = plt.hist(community_size, bins=100)

In [8]:
##    
print('Opening community_text_clean.pkl')
with open("community_text_clean.pkl", 'rb') as f:
    community_text_clean = pickle.load(f)

Opening community_text_clean.pkl


In [2]:
## All abstracts combined for each community 
## {Community_id: <abstract1 abstract2 >,}
## E.g: {0: <Text: structured regression has been successfully used in many...>, 
##      1: <Text: this paper considers the sparse gaussian conditional random...>,
print('Opening text_dict.pkl')
with open("text_dict.pkl", 'rb') as f:
    text_dict = pickle.load(f)

Opening text_dict.pkl


In [ ]:
TF = {}
word_set = set()

overall_freq = Counter()
for i in community_text_clean:
    try:
        fd = nltk.FreqDist(text_dict[i])
        word_set.update(set(list(fd.keys())))
        overall_freq = overall_freq + Counter(fd)
    except:
        print('Didnt work out')
        continue
    TF[i] = overall_freq

In [11]:
def idf(word):
    occ = 0
    for i in TF:
        com = TF.get(i)
        if word in com.keys():
            occ += 1
    if occ == 0:
        occ = 1
    return np.log(10/occ)

In [12]:
def tf_idf(number, com):    
    vec = {}
    # Go through every word 
    for word in com.keys():

        # calculate the term frequency by dividing
        # the occurance of a certain word with the
        # total number of words in the commnity
        tf = TF[number][word]/len(TF[number])
        
        # calculates the inverse document frequency,
        idfreq = idf(word)

        # Then we multiply the two measures 
        val = tf*idfreq

        # save the result in a dictionary with the word
        # as the key and the TF-IDF score as the value
        vec[word] = val
    return vec

In [13]:
tf_idf_all_communities = []
for i in TF:
    v = tf_idf(i, TF.get(i))
    tf_idf_all_communities.append(v)

In [16]:
tf_idf_all_communities[1]

IndexError: list index out of range

In [ ]:
## 
print('Opening TF.pkl')   
with open("TF.pkl", 'rb') as f:
    TF = pickle.load(f)
 
##    
print('Opening TF_IDF.pkl')
with open("tf_idf_all_communities.pkl", 'rb') as f:
    tf_idf_all_communities = pickle.load(f)